In [6]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision.models
from PIL import Image
import numpy as nps
import matplotlib.pyplot as plt
from lib.autoencoder import *
from lib.attention_model import AttentionLayer, MultiAttentionLayer
from lib.correspondence_datamodule import CorrespondenceDataModule
from lib.warping import *
from externals.d2net.lib.utils import *
from lib.loss import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
megadepth_path = "/mnt/c/Users/phill/polybox/Deep Learning/MegaDepthDataset"
data_module = CorrespondenceDataModule(base_path=megadepth_path, batch_size=1)
data_module.prepare_data()
data_module.setup(stage='fit')
dl_train = data_module.train_dataloader()
dl_val = data_module.val_dataloader()



  0%|          | 0/1 [00:00<?, ?it/s]

Building a new training dataset...


  0%|          | 0/1 [00:00<?, ?it/s]

Building the validation dataset...


100%|██████████| 1/1 [00:04<00:00,  4.09s/it]


In [8]:
batch = next(iter(dl_train))


In [9]:
batch["image1"].shape

torch.Size([1, 3, 256, 256])

In [24]:

DistinctivenessLoss()(x1_encoded, x2_encoded, y1, y2, batch)

cpu cpu cpu
cpu cpu cpu


tensor([nan], grad_fn=<AddBackward0>)

In [23]:
x1 = batch['image1']
x2 = batch["image2"]
feature_encoder = FeatureEncoder3().load_from_checkpoint("lightning_logs/version_3/checkpoints/epoch=50-step=7394.ckpt").requires_grad_(False)
attention_model = AttentionLayer(feature_encoder)
attention_model.freeze()
with torch.no_grad():
    x1_encoded = attention_model.concat_layers(attention_model.feature_encoder.forward(x1)) #bx48x32x32
    x2_encoded = attention_model.concat_layers(attention_model.feature_encoder.forward(x2))


print(x1_encoded.shape)
print(x2_encoded.shape)
x1_encoded.requires_grad = False
x2_encoded.requires_grad = False
y1 = attention_model.forward(batch["image1"]) #bx1x32x32
y2 = attention_model.forward(batch["image2"])

print(y1.shape)
print(y2.shape)



# print(x1_encoded[0,0,0,0])

# print(loss)
# print(loss.shape)

torch.Size([1, 96, 16, 16])
torch.Size([1, 96, 16, 16])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])


cuda:0 cuda:0 cuda:0
cuda:0 cuda:0 cuda:0
torch.Size([48, 338]) torch.Size([48, 338]) torch.Size([48, 1024]) torch.Size([48, 1024])
torch.Size([338])


ValueError: only one element tensors can be converted to Python scalars

In [47]:
a = torch.tensor([[0.9041,  0.0196], [-0.3108, -2.4423], [-0.4821,  1.059]])
a.shape

torch.Size([3, 2])